# EBSL + EZKL Pipeline (Complete Implementation)

**Fixed: overflow-safe packing/rebasing + stable product + Jupyter compatibility**

This notebook implements a comprehensive Evidence-Based Subjective Logic (EBSL) pipeline with EZKL zero-knowledge proof generation:

- Single-input ONNX: combined_input = concat(flat(opinions), flat(mask))
- Robust EZKL settings: decomp_legs↑, safe rebasing knobs, version-safe fallbacks
- Stable product via log/exp, sign-preserving denominator clamp
- Async-safe ezkl calls, CLI SRS fallback, verbose timing & run report
- **Jupyter-compatible asyncio handling**

## Key Features
- **Enhanced EBSL Algorithm**: ZK-optimized fusion with overflow protection
- **Property-based Testing**: Hypothesis-driven correctness validation
- **Performance Analysis**: Comparative benchmarking
- **Robust EZKL Integration**: Multiple fallback strategies
- **Jupyter-Compatible**: Fixed asyncio handling for notebook environments

## 1. Setup and Dependencies

In [ ]:
# Install required packages
%pip install torch torchvision
%pip install ezkl
%pip install hypothesis
%pip install matplotlib
%pip install seaborn
%pip install networkx
%pip install plotly
%pip install onnx
%pip install nest_asyncio

In [ ]:
# Configure matplotlib for inline plotting in Jupyter
%matplotlib inline

In [ ]:
import os
import json
import time
import argparse
import traceback
from dataclasses import dataclass, asdict
from contextlib import contextmanager
from typing import Optional, Dict, Any

import numpy as np
import torch
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
from matplotlib.patches import Rectangle, FancyBboxPatch
from matplotlib.collections import LineCollection
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from hypothesis import given, strategies as st, settings as hyp_settings

import asyncio
import inspect
import subprocess
from pathlib import Path as _Path

import ezkl
import onnx

# Set style for better visualizations
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# Configure matplotlib for inline plots
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10
plt.rcParams['axes.titlesize'] = 14
plt.rcParams['axes.labelsize'] = 12

## 2. Logging and Utility Classes

In [ ]:
@dataclass
class StepResult:
    name: str
    ok: bool
    seconds: float
    extra: dict

class Logger:
    def __init__(self, verbose: bool = True):
        self.verbose = verbose
        self.steps = []

    def banner(self, title: str):
        line = "=" * 78
        print(f"\n{line}\n{title}\n{line}")

    def info(self, msg: str):
        if self.verbose:
            print(msg)

    def warn(self, msg: str):
        print(f"[!] {msg}")

    def error(self, msg: str):
        print(f"[✗] {msg}")

    def ok(self, msg: str):
        print(f"[✓] {msg}")

    @contextmanager
    def timed(self, name: str, extra: Optional[Dict[str, Any]] = None):
        start = time.perf_counter()
        ok = True
        info = dict(extra or {})
        try:
            yield info
            ok = True
        except Exception as e:
            ok = False
            info["exception"] = repr(e)
            info["traceback"] = traceback.format_exc(limit=12)
            self.error(f"{name} failed: {e}")
            raise
        finally:
            dur = time.perf_counter() - start
            self.steps.append(StepResult(name, ok, dur, info))
            status = "OK" if ok else "FAIL"
            self.info(f"[{status}] {name} in {dur:.3f}s")

    def dump_report(self, path: str):
        os.makedirs(os.path.dirname(path), exist_ok=True)
        with open(path, "w") as f:
            json.dump([asdict(s) for s in self.steps], f, indent=2)
        self.ok(f"Run report written: {path}")

## 3. Visualization Utilities

In [ ]:
class EBSLVisualizer:
    """
    Comprehensive visualization toolkit for EBSL pipeline components
    """
    
    @staticmethod
    def create_network_topology(num_nodes=10, trust_matrix=None, title="Trust Network Topology"):
        """
        Create a network topology visualization showing trust relationships
        """
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 8))
        
        # Generate or use provided trust matrix
        if trust_matrix is None:
            # Create a realistic trust matrix
            np.random.seed(42)
            trust_matrix = np.random.beta(2, 5, (num_nodes, num_nodes))
            np.fill_diagonal(trust_matrix, 1.0)  # Self-trust = 1
        
        # Create network graph
        G = nx.DiGraph()
        positions = nx.circular_layout(range(num_nodes))
        
        # Add nodes with reputation-based sizing
        for i in range(num_nodes):
            reputation = np.mean(trust_matrix[:, i])  # Average incoming trust
            G.add_node(i, reputation=reputation)
        
        # Add edges with trust-based coloring
        threshold = 0.3  # Minimum trust to show edge
        for i in range(num_nodes):
            for j in range(num_nodes):
                if i != j and trust_matrix[i, j] > threshold:
                    G.add_edge(i, j, weight=trust_matrix[i, j])
        
        # Plot network graph
        node_sizes = [G.nodes[i]['reputation'] * 1000 for i in G.nodes()]
        edge_weights = [G[u][v]['weight'] for u, v in G.edges()]
        
        nx.draw_networkx_nodes(G, positions, node_size=node_sizes, 
                              node_color=[G.nodes[i]['reputation'] for i in G.nodes()],
                              cmap='viridis', alpha=0.8, ax=ax1)
        nx.draw_networkx_edges(G, positions, width=[w*3 for w in edge_weights],
                              edge_color=edge_weights, edge_cmap=plt.cm.plasma,
                              alpha=0.6, arrows=True, arrowsize=20, ax=ax1)
        nx.draw_networkx_labels(G, positions, ax=ax1)
        
        ax1.set_title(f"{title}\n(Node size ∝ reputation, Edge color/width ∝ trust)")
        ax1.axis('off')
        
        # Plot trust matrix heatmap
        im = ax2.imshow(trust_matrix, cmap='RdYlBu_r', aspect='auto')
        ax2.set_title("Trust Matrix Heatmap")
        ax2.set_xlabel("Trusted Agent")
        ax2.set_ylabel("Trusting Agent")
        
        # Add colorbar
        plt.colorbar(im, ax=ax2, label='Trust Level')
        
        plt.tight_layout()
        plt.show()
        return trust_matrix
    
    @staticmethod
    def visualize_opinion_components(opinions_tensor, title="EBSL Opinion Components"):
        """
        Visualize belief, disbelief, uncertainty, and base rate components
        """
        fig, axes = plt.subplots(2, 2, figsize=(15, 12))
        
        # Extract components
        beliefs = opinions_tensor[:, 0].detach().numpy()
        disbeliefs = opinions_tensor[:, 1].detach().numpy()
        uncertainties = opinions_tensor[:, 2].detach().numpy()
        base_rates = opinions_tensor[:, 3].detach().numpy()
        
        # Plot distributions
        axes[0, 0].hist(beliefs, bins=20, alpha=0.7, color='green', edgecolor='black')
        axes[0, 0].set_title('Belief Distribution')
        axes[0, 0].set_xlabel('Belief Value')
        axes[0, 0].set_ylabel('Frequency')
        axes[0, 0].axvline(np.mean(beliefs), color='red', linestyle='--', label=f'Mean: {np.mean(beliefs):.3f}')
        axes[0, 0].legend()
        
        axes[0, 1].hist(disbeliefs, bins=20, alpha=0.7, color='red', edgecolor='black')
        axes[0, 1].set_title('Disbelief Distribution')
        axes[0, 1].set_xlabel('Disbelief Value')
        axes[0, 1].set_ylabel('Frequency')
        axes[0, 1].axvline(np.mean(disbeliefs), color='blue', linestyle='--', label=f'Mean: {np.mean(disbeliefs):.3f}')
        axes[0, 1].legend()
        
        axes[1, 0].hist(uncertainties, bins=20, alpha=0.7, color='orange', edgecolor='black')
        axes[1, 0].set_title('Uncertainty Distribution')
        axes[1, 0].set_xlabel('Uncertainty Value')
        axes[1, 0].set_ylabel('Frequency')
        axes[1, 0].axvline(np.mean(uncertainties), color='purple', linestyle='--', label=f'Mean: {np.mean(uncertainties):.3f}')
        axes[1, 0].legend()
        
        axes[1, 1].hist(base_rates, bins=20, alpha=0.7, color='blue', edgecolor='black')
        axes[1, 1].set_title('Base Rate Distribution')
        axes[1, 1].set_xlabel('Base Rate Value')
        axes[1, 1].set_ylabel('Frequency')
        axes[1, 1].axvline(np.mean(base_rates), color='green', linestyle='--', label=f'Mean: {np.mean(base_rates):.3f}')
        axes[1, 1].legend()
        
        plt.suptitle(title, fontsize=16)
        plt.tight_layout()
        plt.show()
        
        # Create a combined ternary-like plot
        fig, ax = plt.subplots(figsize=(10, 8))
        
        # Scatter plot of belief vs disbelief, colored by uncertainty
        scatter = ax.scatter(beliefs, disbeliefs, c=uncertainties, s=100, 
                           cmap='viridis', alpha=0.7, edgecolors='black')
        ax.set_xlabel('Belief')
        ax.set_ylabel('Disbelief')
        ax.set_title('Opinion Space (Belief vs Disbelief, colored by Uncertainty)')
        
        # Add constraint line (b + d + u = 1)
        x = np.linspace(0, 1, 100)
        for u_val in np.linspace(0, 1, 6):
            y = 1 - u_val - x
            mask = (y >= 0) & (y <= 1)
            ax.plot(x[mask], y[mask], '--', alpha=0.3, label=f'u={u_val:.1f}' if u_val in [0, 0.5, 1] else None)
        
        plt.colorbar(scatter, label='Uncertainty')
        ax.legend()
        plt.show()
    
    @staticmethod
    def visualize_zk_circuit_concept(title="ZK Circuit Structure Concept"):
        """
        Visualize the conceptual structure of the ZK circuit
        """
        fig, ax = plt.subplots(figsize=(14, 10))
        
        # Define circuit components
        components = {
            'input': {'pos': (1, 8), 'size': (2, 1), 'color': 'lightblue', 'label': 'Private Input\n(Opinions + Mask)'},
            'unpack': {'pos': (4, 8), 'size': (2, 1), 'color': 'lightgreen', 'label': 'Unpack & Validate\nOpinions'},
            'weight': {'pos': (7, 9), 'size': (2, 0.8), 'color': 'yellow', 'label': 'Weight\nCalculation'},
            'fusion': {'pos': (7, 7), 'size': (2, 0.8), 'color': 'orange', 'label': 'EBSL Fusion\nAlgorithm'},
            'reputation': {'pos': (10, 8), 'size': (2, 1), 'color': 'pink', 'label': 'Reputation\nCalculation'},
            'output': {'pos': (13, 8), 'size': (2, 1), 'color': 'lightcoral', 'label': 'Public Output\n(Reputation Score)'}
        }
        
        # Draw components
        for name, comp in components.items():
            rect = FancyBboxPatch(
                comp['pos'], comp['size'][0], comp['size'][1],
                boxstyle="round,pad=0.1",
                facecolor=comp['color'],
                edgecolor='black',
                linewidth=2
            )
            ax.add_patch(rect)
            
            # Add label
            ax.text(comp['pos'][0] + comp['size'][0]/2, comp['pos'][1] + comp['size'][1]/2,
                   comp['label'], ha='center', va='center', fontsize=9, weight='bold')
        
        # Draw connections
        connections = [
            ('input', 'unpack'),
            ('unpack', 'weight'),
            ('unpack', 'fusion'),
            ('weight', 'reputation'),
            ('fusion', 'reputation'),
            ('reputation', 'output')
        ]
        
        for start, end in connections:
            start_pos = components[start]['pos']
            end_pos = components[end]['pos']
            start_x = start_pos[0] + components[start]['size'][0]
            start_y = start_pos[1] + components[start]['size'][1]/2
            end_x = end_pos[0]
            end_y = end_pos[1] + components[end]['size'][1]/2
            
            ax.arrow(start_x, start_y, end_x - start_x - 0.1, end_y - start_y,
                    head_width=0.2, head_length=0.1, fc='black', ec='black')
        
        # Add ZK properties annotations
        ax.text(8, 5, '🔒 Zero-Knowledge Properties:', fontsize=12, weight='bold')
        ax.text(8, 4.5, '• Input opinions remain private', fontsize=10)
        ax.text(8, 4.1, '• Computation is verifiable', fontsize=10)
        ax.text(8, 3.7, '• No trust in prover required', fontsize=10)
        ax.text(8, 3.3, '• Succinctly verifiable proof', fontsize=10)
        
        # Add circuit complexity info
        ax.text(8, 2.5, '⚡ Circuit Complexity:', fontsize=12, weight='bold')
        ax.text(8, 2.1, '• Arithmetic operations only', fontsize=10)
        ax.text(8, 1.7, '• Fixed-size constraint system', fontsize=10)
        ax.text(8, 1.3, '• Overflow-safe operations', fontsize=10)
        
        ax.set_xlim(0, 16)
        ax.set_ylim(0, 10)
        ax.set_title(title, fontsize=16, weight='bold')
        ax.axis('off')
        
        plt.tight_layout()
        plt.show()
    
    @staticmethod
    def plot_fusion_process(opinions_before, opinion_after, title="EBSL Fusion Process"):
        """
        Visualize the opinion fusion process
        """
        fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(18, 6))
        
        # Before fusion - individual opinions
        components = ['Belief', 'Disbelief', 'Uncertainty', 'Base Rate']
        x = np.arange(len(components))
        width = 0.8 / len(opinions_before)
        
        for i, opinion in enumerate(opinions_before):
            offset = (i - len(opinions_before)/2 + 0.5) * width
            ax1.bar(x + offset, opinion.detach().numpy(), width, 
                   label=f'Agent {i+1}', alpha=0.7)
        
        ax1.set_xlabel('Opinion Components')
        ax1.set_ylabel('Value')
        ax1.set_title('Before Fusion: Individual Opinions')
        ax1.set_xticks(x)
        ax1.set_xticklabels(components)
        ax1.legend()
        ax1.grid(True, alpha=0.3)
        
        # After fusion - fused opinion
        ax2.bar(components, opinion_after.detach().numpy(), 
               color=['green', 'red', 'orange', 'blue'], alpha=0.7)
        ax2.set_xlabel('Opinion Components')
        ax2.set_ylabel('Value')
        ax2.set_title('After Fusion: Aggregated Opinion')
        ax2.grid(True, alpha=0.3)
        
        # Add values on bars
        for i, v in enumerate(opinion_after.detach().numpy()):
            ax2.text(i, v + 0.01, f'{v:.3f}', ha='center', va='bottom')
        
        # Comparison radar chart
        angles = np.linspace(0, 2 * np.pi, len(components), endpoint=False)
        angles = np.concatenate((angles, [angles[0]]))  # Complete the circle
        
        # Plot individual opinions
        for i, opinion in enumerate(opinions_before[:3]):  # Show only first 3 for clarity
            values = opinion.detach().numpy().tolist()
            values += [values[0]]  # Complete the circle
            ax3.plot(angles, values, 'o-', linewidth=2, label=f'Agent {i+1}', alpha=0.7)
        
        # Plot fused opinion
        fused_values = opinion_after.detach().numpy().tolist()
        fused_values += [fused_values[0]]
        ax3.plot(angles, fused_values, 'o-', linewidth=3, label='Fused', color='black')
        ax3.fill(angles, fused_values, alpha=0.25, color='black')
        
        ax3.set_xticks(angles[:-1])
        ax3.set_xticklabels(components)
        ax3.set_ylim(0, 1)
        ax3.set_title('Radar Comparison')
        ax3.legend()
        ax3.grid(True)
        
        plt.suptitle(title, fontsize=16)
        plt.tight_layout()
        plt.show()
    
    @staticmethod
    def plot_performance_comparison(classical_times, zk_times, opinion_counts, title="Performance Comparison"):
        """
        Plot performance comparison between classical and ZK-friendly algorithms
        """
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
        
        # Execution time comparison
        ax1.plot(opinion_counts, classical_times, 'o-', label='Classical EBSL', linewidth=2, markersize=8)
        ax1.plot(opinion_counts, zk_times, 's-', label='ZK-Friendly EBSL', linewidth=2, markersize=8)
        ax1.set_xlabel('Number of Opinions')
        ax1.set_ylabel('Execution Time (seconds)')
        ax1.set_title('Execution Time vs Number of Opinions')
        ax1.legend()
        ax1.grid(True, alpha=0.3)
        ax1.set_yscale('log')
        
        # Relative performance (speedup/slowdown)
        relative_perf = np.array(classical_times) / np.array(zk_times)
        ax2.plot(opinion_counts, relative_perf, 'o-', color='purple', linewidth=2, markersize=8)
        ax2.axhline(y=1, color='red', linestyle='--', alpha=0.7, label='Equal Performance')
        ax2.set_xlabel('Number of Opinions')
        ax2.set_ylabel('Relative Performance (Classical/ZK-Friendly)')
        ax2.set_title('Relative Performance Ratio')
        ax2.legend()
        ax2.grid(True, alpha=0.3)
        
        # Add annotations for significant points
        max_speedup_idx = np.argmax(relative_perf)
        ax2.annotate(f'Max ratio: {relative_perf[max_speedup_idx]:.2f}x\nat {opinion_counts[max_speedup_idx]} opinions',
                    xy=(opinion_counts[max_speedup_idx], relative_perf[max_speedup_idx]),
                    xytext=(10, 10), textcoords='offset points',
                    bbox=dict(boxstyle='round,pad=0.5', fc='yellow', alpha=0.7),
                    arrowprops=dict(arrowstyle='->', connectionstyle='arc3,rad=0'))
        
        plt.suptitle(title, fontsize=16)
        plt.tight_layout()
        plt.show()
    
    @staticmethod
    def plot_reputation_evolution(reputation_scores, agents, title="Reputation Score Evolution"):
        """
        Plot how reputation scores evolve or are distributed
        """
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
        
        # Reputation distribution
        ax1.hist(reputation_scores, bins=20, alpha=0.7, color='skyblue', edgecolor='black')
        ax1.axvline(np.mean(reputation_scores), color='red', linestyle='--', 
                   label=f'Mean: {np.mean(reputation_scores):.3f}')
        ax1.axvline(np.median(reputation_scores), color='green', linestyle='--', 
                   label=f'Median: {np.median(reputation_scores):.3f}')
        ax1.set_xlabel('Reputation Score')
        ax1.set_ylabel('Frequency')
        ax1.set_title('Reputation Score Distribution')
        ax1.legend()
        ax1.grid(True, alpha=0.3)
        
        # Individual agent reputation
        colors = plt.cm.Set3(np.linspace(0, 1, len(agents)))
        bars = ax2.bar(range(len(agents)), reputation_scores, color=colors, alpha=0.7, edgecolor='black')
        ax2.set_xlabel('Agent ID')
        ax2.set_ylabel('Reputation Score')
        ax2.set_title('Individual Agent Reputation Scores')
        ax2.set_xticks(range(len(agents)))
        ax2.set_xticklabels([f'Agent {i}' for i in agents])
        ax2.grid(True, alpha=0.3)
        
        # Add value labels on bars
        for bar, score in zip(bars, reputation_scores):
            height = bar.get_height()
            ax2.text(bar.get_x() + bar.get_width()/2., height + 0.01,
                    f'{score:.3f}', ha='center', va='bottom', fontsize=9)
        
        plt.suptitle(title, fontsize=16)
        plt.tight_layout()
        plt.show()

# Initialize the visualizer
visualizer = EBSLVisualizer()

## 4. Jupyter-Compatible Async Helpers

In [ ]:
def run_with_loop(func, /, *args, **kwargs):
    """
    Jupyter-compatible async execution.
    Handles both sync and async functions, with fallbacks for different environments.
    """
    # First try direct execution (for sync functions)
    result = func(*args, **kwargs)
    
    # If it's awaitable, we need special handling
    if inspect.isawaitable(result):
        try:
            # Try nest_asyncio for Jupyter compatibility
            import nest_asyncio
            nest_asyncio.apply()
            return asyncio.run(result)
        except ImportError:
            # Fallback: thread-based execution
            import concurrent.futures
            
            def run_async():
                loop = asyncio.new_event_loop()
                asyncio.set_event_loop(loop)
                try:
                    return loop.run_until_complete(result)
                finally:
                    loop.close()
            
            with concurrent.futures.ThreadPoolExecutor() as executor:
                return executor.submit(run_async).result()
        except Exception as e:
            # Last resort: try to get the event loop and run
            try:
                loop = asyncio.get_event_loop()
                if loop.is_running():
                    # Create a task and let it run
                    import concurrent.futures
                    
                    def run_in_thread():
                        new_loop = asyncio.new_event_loop()
                        asyncio.set_event_loop(new_loop)
                        try:
                            return new_loop.run_until_complete(result)
                        finally:
                            new_loop.close()
                    
                    with concurrent.futures.ThreadPoolExecutor() as executor:
                        return executor.submit(run_in_thread).result()
                else:
                    return loop.run_until_complete(result)
            except:
                # Absolute fallback - might fail in some cases
                return asyncio.run(result)
    
    return result

def get_srs_with_fallback(settings_path: str, srs_path: str, logger: Logger) -> bool:
    """
    Try Python binding; if that fails, fall back to `ezkl get-srs -S settings.json`.
    """
    try:
        ok = run_with_loop(ezkl.get_srs, srs_path=srs_path, settings_path=settings_path)
        if ok:
            return True
        logger.warn("ezkl.get_srs returned False; attempting CLI fallback")
    except Exception as e:
        logger.warn(f"ezkl.get_srs raised {e!r}; attempting CLI fallback")

    try:
        cmd = ["ezkl", "get-srs", "-S", settings_path]
        subprocess.run(cmd, check=True)
        if _Path(srs_path).exists():
            return True
        default_srs = _Path.home() / ".ezkl" / "srs" / "kzg15.srs"
        if default_srs.exists():
            import shutil
            shutil.copy2(default_srs, srs_path)
            logger.info(f"Copied SRS from default location to {srs_path}")
            return True
        return False
    except Exception as e2:
        logger.error(f"CLI get-srs failed: {e2!r}")
        return False

## 5. EZKL Safe Utilities

In [ ]:
def safe_setattr(obj, name, value, logger: Logger):
    """Set attribute if it exists in bindings; otherwise log and continue."""
    try:
        setattr(obj, name, value)
        logger.info(f"run_args.{name} = {value!r}")
    except Exception as e:
        logger.warn(f"run_args field {name!r} not supported by this ezkl version: {e!r}")

def safe_calibrate(logger: Logger, *, data, model, settings, **kwargs) -> bool:
    """
    Call ezkl.calibrate_settings with rich kwargs; on TypeError,
    progressively back off to a minimal call.
    """
    try:
        return bool(run_with_loop(ezkl.calibrate_settings, data=data, model=model, settings=settings, **kwargs))
    except TypeError as e:
        logger.warn(f"Calibration kwargs not fully supported ({e}); retrying with reduced kwargs")
        # Fallback 1: keep impactful knobs
        try_kwargs = {k: kwargs[k] for k in ["target", "lookup_safety_margin", "max_logrows"] if k in kwargs}
        try:
            return bool(run_with_loop(ezkl.calibrate_settings, data=data, model=model, settings=settings, **try_kwargs))
        except TypeError as e2:
            logger.warn(f"Reduced calibration call failed ({e2}); retrying minimal")
            # Fallback 2: minimal signature
            return bool(run_with_loop(ezkl.calibrate_settings, data=data, model=model, settings=settings))

def summarize_settings(path: str) -> dict:
    try:
        with open(path, "r") as f:
            s = json.load(f)
        out = {
            "logrows": s.get("logrows"),
            "input_visibility": s.get("input_visibility"),
            "param_visibility": s.get("param_visibility"),
            "output_visibility": s.get("output_visibility"),
        }
        if all(v is None for v in out.values()) and isinstance(s, dict):
            run_args = s.get("run_args") or s.get("py_run_args") or {}
            out.update({
                "input_visibility": run_args.get("input_visibility", out["input_visibility"]),
                "param_visibility": run_args.get("param_visibility", out["param_visibility"]),
                "output_visibility": run_args.get("output_visibility", out["output_visibility"]),
            })
        return out
    except Exception:
        return {}

## 6. EBSL Algorithm Implementation

In [ ]:
class ClassicalEBSLAlgorithm:
    @staticmethod
    def fuse(opinions_tensor: torch.Tensor) -> torch.Tensor:
        # opinions_tensor: [N, 4] -> [b,d,u,a]
        b, d, u, a = [o.squeeze(-1) for o in opinions_tensor.split(1, dim=-1)]
        denominator = torch.sum(u, dim=-1) - (opinions_tensor.shape[0] - 1)
        if torch.any(denominator == 0):
            denominator = denominator + (denominator == 0) * 1e-9
        b_fused = torch.sum(b * u, dim=-1) / denominator
        d_fused = torch.sum(d * u, dim=-1) / denominator
        u_fused = torch.prod(u, dim=-1) / denominator
        a_fused = torch.sum((a * u), dim=-1) / denominator
        return torch.stack([b_fused, d_fused, u_fused, a_fused], dim=-1)

class EBSLAlgorithm:
    @staticmethod
    def fuse(opinions_tensor: torch.Tensor) -> torch.Tensor:
        b, d, u, a = [o.squeeze(-1) for o in opinions_tensor.split(1, dim=-1)]
        denominator = torch.sum(u, dim=-1) - (opinions_tensor.shape[0] - 1)
        is_zero = (denominator == 0).to(torch.float32)
        denominator = denominator + (is_zero * 1e-9)
        inv_denominator = torch.reciprocal(denominator)
        b_fused = torch.sum(b * u, dim=-1) * inv_denominator
        d_fused = torch.sum(d * u, dim=-1) * inv_denominator
        u_fused = torch.prod(u, dim=-1) * inv_denominator
        a_fused = torch.sum((a * u), dim=-1) * inv_denominator
        return torch.stack([b_fused, d_fused, u_fused, a_fused], dim=-1)

    @staticmethod
    def calculate_reputation(final_opinion_tensor: torch.Tensor) -> torch.Tensor:
        b, d, u, a = [o.squeeze(-1) for o in final_opinion_tensor.split(1, dim=-1)]
        return b + a * u

## 7. Property-Based Testing and Performance Analysis

In [ ]:
@st.composite
def opinion_strategy(draw):
    b = draw(st.floats(0.0, 1.0))
    d = draw(st.floats(0.0, 1.0 - b))
    u = 1.0 - b - d
    a = draw(st.floats(0.0, 1.0))
    return torch.tensor([b, d, u, a], dtype=torch.float32)

@st.composite
def opinions_tensor_strategy(draw, min_opinions=2, max_opinions=50):
    num_opinions = draw(st.integers(min_opinions, max_opinions))
    opinions = draw(st.lists(opinion_strategy(), min_size=num_opinions, max_size=num_opinions))
    return torch.stack(opinions)

def run_property_based_correctness_test(logger: Logger):
    logger.banner("Property-based correctness test")
    @given(opinions_tensor=opinions_tensor_strategy())
    @hyp_settings(max_examples=100, deadline=None)
    def test_fusion_equivalence(opinions_tensor):
        classical_result = ClassicalEBSLAlgorithm.fuse(opinions_tensor)
        zk_friendly_result = EBSLAlgorithm.fuse(opinions_tensor)
        assert torch.allclose(classical_result, zk_friendly_result, atol=1e-6)
    with logger.timed("hypothesis_equivalence_test"):
        test_fusion_equivalence()
    logger.ok("Equivalence holds for 100 random examples")

def run_comparative_performance_analysis(logger: Logger, skip_plots: bool = False):
    logger.banner("Comparative performance analysis")
    opinion_counts = list(range(10, 201, 10))
    results = {'classical': [], 'zk_friendly': []}
    
    # Generate sample data for visualizations
    sample_opinions_tensor = None
    
    with logger.timed("perf_benchmark"):
        for count in opinion_counts:
            b = torch.rand(count)
            d = torch.rand(count) * (1 - b)
            u = 1 - b - d
            a = torch.rand(count)
            sample_tensor = torch.stack([b, d, u, a], dim=1)
            
            # Save a sample for visualization
            if count == 50:  # Use medium-sized sample for visualization
                sample_opinions_tensor = sample_tensor.clone()
            
            t0 = time.perf_counter()
            ClassicalEBSLAlgorithm.fuse(sample_tensor)
            results['classical'].append(time.perf_counter() - t0)
            t0 = time.perf_counter()
            EBSLAlgorithm.fuse(sample_tensor)
            results['zk_friendly'].append(time.perf_counter() - t0)
    
    logger.ok("Perf benchmark complete")
    
    if not skip_plots:
        os.makedirs("zkml_artifacts", exist_ok=True)
        
        # Original performance plot
        plt.figure(figsize=(10, 6))
        plt.plot(opinion_counts, results['classical'], marker='o', label='Classical')
        plt.plot(opinion_counts, results['zk_friendly'], marker='x', label='ZK-Friendly')
        plt.xlabel("Number of Opinions to Fuse")
        plt.ylabel("Execution Time [s] (log)")
        plt.title("Performance Comparison: Classical vs ZK-Friendly EBSL Fusion")
        plt.legend(); plt.grid(True); plt.yscale('log'); plt.tight_layout()
        plot_path = os.path.join("zkml_artifacts", "perf_plot.png")
        plt.savefig(plot_path, dpi=160)
        plt.show()
        logger.ok(f"Saved performance plot: {plot_path}")
        
        # Enhanced performance visualization using our visualizer
        visualizer.plot_performance_comparison(
            results['classical'], 
            results['zk_friendly'], 
            opinion_counts,
            "Enhanced Performance Analysis: Classical vs ZK-Friendly EBSL"
        )
        
        # Visualize opinion components from sample data
        if sample_opinions_tensor is not None:
            logger.info("Generating opinion component visualizations...")
            visualizer.visualize_opinion_components(
                sample_opinions_tensor,
                "Sample Opinion Components Analysis (50 agents)"
            )
            
            # Show fusion process with a subset of opinions
            subset_opinions = sample_opinions_tensor[:5]  # Use first 5 for clarity
            fused_opinion = EBSLAlgorithm.fuse(subset_opinions)
            visualizer.plot_fusion_process(
                subset_opinions,
                fused_opinion,
                "EBSL Fusion Process Demonstration (5 agents)"
            )
    
    return results, sample_opinions_tensor

## 8. ZK-Optimized EBSL Module

In [ ]:
class EBslFusionModule(torch.nn.Module):
    """
    ZK-optimized EBSL fusion module with overflow-safe ops.
    Input: combined tensor with opinions and mask flattened and concatenated
    Outputs:
      fused:    (B, 4)
      rep:      (B, 1)
    """
    def __init__(self, max_opinions: int = 16):
        super().__init__()
        self.max_opinions = max_opinions
        self.opinions_size = max_opinions * 4  # N * 4 for [b,d,u,a]
        self.mask_size = max_opinions         # N for mask
        self.register_buffer('epsilon', torch.tensor(1e-6))
        self.register_buffer('one', torch.tensor(1.0))

    def forward(self, combined_input: torch.Tensor):
        batch_size = combined_input.shape[0]

        # Split & reshape back to opinions and mask
        opinions_flat = combined_input[:, :self.opinions_size]
        mask_flat = combined_input[:, self.opinions_size:self.opinions_size + self.mask_size]
        opinions = opinions_flat.view(batch_size, self.max_opinions, 4)
        mask = mask_flat.view(batch_size, self.max_opinions)

        b = opinions[..., 0]
        d = opinions[..., 1]
        u = opinions[..., 2]
        a = opinions[..., 3]

        m = mask
        K = torch.sum(m, dim=1)

        sum_bu = torch.sum((b * u) * m, dim=1)
        sum_du = torch.sum((d * u) * m, dim=1)
        sum_au = torch.sum((a * u) * m, dim=1)
        sum_u  = torch.sum(u * m, dim=1)

        # Stable product via logs (avoids huge intermediates)
        u_masked = u * m + (self.one - m)                # 1 for masked-out entries
        u_clamped = torch.clamp(u_masked, min=self.epsilon, max=self.one)
        sum_log = torch.sum(torch.log(u_clamped), dim=1)
        prod_u = torch.exp(sum_log)

        # Sign-preserving denom clamp
        denom = sum_u - K + self.one
        denom_sign = torch.where(denom >= 0, self.one, -self.one)
        denom = denom_sign * torch.clamp(torch.abs(denom), min=self.epsilon)

        b_f = sum_bu / denom
        d_f = sum_du / denom
        u_f = prod_u / denom
        a_f = sum_au / denom

        fused = torch.stack([b_f, d_f, u_f, a_f], dim=1)
        rep = (b_f + a_f * u_f).unsqueeze(1)
        return fused, rep

def _gen_synthetic_opinions(N: int):
    b = torch.rand(N)
    d = torch.rand(N) * (1.0 - b)
    u = 1.0 - b - d
    a = torch.rand(N)
    opinions = torch.stack([b, d, u, a], dim=1)
    mask = torch.ones(N)
    return opinions, mask

## 9. ZKML Pipeline Implementation

In [ ]:
def run_zkml_pipeline_with_ebsl(logger: Logger, max_opinions: int = 16,
                               zk_strategy: str = "balanced",
                               manual_input_scale: int = None,
                               manual_param_scale: int = None,
                               skip_calibration: bool = False):
    logger.banner("ZKML pipeline: EBSL fusion in EZKL")
    wd = os.path.abspath("zkml_artifacts")
    os.makedirs(wd, exist_ok=True)

    # 1) Export ONNX
    with logger.timed("export_onnx", extra={"max_opinions": max_opinions}) as info:
        model = EBslFusionModule(max_opinions=max_opinions).eval()
        opinions, mask = _gen_synthetic_opinions(max_opinions)
        opinions_b, mask_b = opinions.unsqueeze(0), mask.unsqueeze(0)

        # Create combined input for single-input model
        opinions_flat = opinions_b.flatten(start_dim=1)
        mask_flat = mask_b.flatten(start_dim=1)
        combined_input = torch.cat([opinions_flat, mask_flat], dim=1)

        onnx_path = os.path.join(wd, "ebsl_model.onnx")
        torch.onnx.export(
            model,
            combined_input,
            onnx_path,
            input_names=["combined_input"],
            output_names=["fused", "rep"],
            opset_version=13,
            dynamic_axes=None,
        )
        info["onnx_path"] = onnx_path
        logger.ok(f"Exported ONNX -> {onnx_path}")
        if logger.verbose:
            logger.info(f"combined input shape: {tuple(combined_input.shape)}")
            logger.info("sample opinions[0,:3]: " + json.dumps(opinions_b[0, :3].detach().numpy().tolist(), indent=2))

    # 2) gen_settings
    with logger.timed("gen_settings") as info:
        run_args = ezkl.PyRunArgs()
        # visibility
        safe_setattr(run_args, "input_visibility",  "public", logger)
        safe_setattr(run_args, "param_visibility",  "fixed",  logger)
        safe_setattr(run_args, "output_visibility", "public", logger)

        # packing & rebasing defenses
        safe_setattr(run_args, "decomp_base", 16384, logger)          # 2^14 base
        safe_setattr(run_args, "decomp_legs", 4, logger)              # 4 limbs => ~56-bit capacity
        safe_setattr(run_args, "div_rebasing", True, logger)          # may be absent; safe_setattr handles it
        safe_setattr(run_args, "scale_rebase_multiplier", 2, logger)
        safe_setattr(run_args, "check_mode", "safe", logger)

        # scales
        if manual_input_scale is not None:
            safe_setattr(run_args, "input_scale", manual_input_scale, logger)
            safe_setattr(run_args, "param_scale", manual_param_scale or manual_input_scale, logger)
            logger.info(f"Using manual scales: input={manual_input_scale}, param={manual_param_scale or manual_input_scale}")
        elif zk_strategy == "conservative":
            safe_setattr(run_args, "input_scale", 4, logger)
            safe_setattr(run_args, "param_scale", 4, logger)
            logger.info("Using conservative ZK strategy (scale=4)")
        elif zk_strategy == "aggressive":
            safe_setattr(run_args, "input_scale", 8, logger)
            safe_setattr(run_args, "param_scale", 8, logger)
            logger.info("Using aggressive ZK strategy (scale=8)")
        else:  # balanced
            safe_setattr(run_args, "input_scale", 6, logger)
            safe_setattr(run_args, "param_scale", 6, logger)
            logger.info("Using balanced ZK strategy (scale=6)")

        settings_path = os.path.join(wd, "settings.json")
        # BUGFIX: use the outer-scope onnx_path, not info["onnx_path"]
        ok = run_with_loop(ezkl.gen_settings, model=onnx_path, output=settings_path, py_run_args=run_args)
        if not ok:
            raise RuntimeError("gen_settings failed")
        info["settings_path"] = settings_path
        info["summary"] = summarize_settings(settings_path)
        logger.ok(f"Generated settings -> {settings_path}")
        logger.info("settings summary: " + json.dumps(info["summary"], indent=2))

    return onnx_path, settings_path  # Return for continuation

## 10. Complete Pipeline Continuation Functions

In [ ]:
def continue_zkml_pipeline(logger: Logger, onnx_path: str, settings_path: str, 
                          max_opinions: int = 16, skip_calibration: bool = False):
    """
    Continue the ZKML pipeline from where run_zkml_pipeline_with_ebsl left off.
    """
    wd = os.path.dirname(settings_path)
    
    # 3) input.json (GraphData: single input vector)
    with logger.timed("write_input_json") as info:
        opinions, mask = _gen_synthetic_opinions(max_opinions)
        opinions_b, mask_b = opinions.unsqueeze(0), mask.unsqueeze(0)
        opinions_data = opinions_b.detach().cpu().numpy().flatten().tolist()
        mask_data = mask_b.detach().cpu().numpy().flatten().tolist()
        combined_input_list = opinions_data + mask_data

        graph_data = {
            "input_data": [combined_input_list],
            "input_shapes": [[len(combined_input_list)]]
        }

        input_json = os.path.join(wd, "input.json")
        with open(input_json, "w") as f:
            json.dump(graph_data, f)
        info["input_json"] = input_json
        logger.ok(f"Wrote input -> {input_json}")

    # 4) calibrate_settings (robust)
    with logger.timed("calibrate_settings") as info:
        if skip_calibration:
            logger.info("Skipping calibration (skip_calibration=True)")
            info["calibrated"] = False
            info["skipped"] = True
        else:
            cal_kwargs = dict(
                target="resources",
                lookup_safety_margin=2,
                scales=[6, 8, 10, 12],
                scale_rebase_multiplier=[1, 2, 4],
                max_logrows=16,
            )
            try:
                cal_ok = safe_calibrate(logger, data=input_json, model=onnx_path, settings=settings_path, **cal_kwargs)
                info["calibrated"] = bool(cal_ok)
                if cal_ok:
                    logger.ok("Settings calibrated successfully")
                else:
                    logger.warn("Calibration returned False - using fallback settings")
            except Exception as e:
                info["calibration_exception"] = repr(e)
                logger.warn(f"Calibration failed: {e}")

    # 5) compile_circuit
    with logger.timed("compile_circuit") as info:
        compiled_path = os.path.join(wd, "compiled.onnx")
        ok = run_with_loop(ezkl.compile_circuit, model=onnx_path, compiled_circuit=compiled_path, settings_path=settings_path)
        if not ok:
            raise RuntimeError("compile_circuit failed")
        info["compiled_path"] = compiled_path
        logger.ok(f"Compiled circuit -> {compiled_path}")
        try:
            circuit_size = os.path.getsize(compiled_path)
            info["circuit_size_bytes"] = circuit_size
            logger.info(f"Compiled circuit size: {circuit_size:,} bytes ({circuit_size/1024:.1f} KB)")
        except Exception:
            pass

    # 6) get_srs
    with logger.timed("get_srs") as info:
        srs_path = os.path.join(wd, "kzg.srs")
        ok = get_srs_with_fallback(settings_path=settings_path, srs_path=srs_path, logger=logger)
        if not ok:
            raise RuntimeError("get_srs failed")
        info["srs_path"] = srs_path
        logger.ok(f"SRS ready -> {srs_path}")

    return compiled_path, srs_path, input_json

def complete_proof_generation(logger: Logger, compiled_path: str, srs_path: str, 
                             input_json: str, settings_path: str, max_opinions: int = 16):
    """
    Complete the proof generation phase.
    """
    wd = os.path.dirname(compiled_path)
    
    # 7) setup
    with logger.timed("setup") as info:
        pk_path = os.path.join(wd, "model.pk")
        vk_path = os.path.join(wd, "model.vk")
        ok = run_with_loop(ezkl.setup, model=compiled_path, vk_path=vk_path, pk_path=pk_path, srs_path=srs_path)
        if not ok:
            raise RuntimeError("setup failed")
        info["pk_path"] = pk_path
        info["vk_path"] = vk_path
        logger.ok(f"Setup complete -> pk:{pk_path}, vk:{vk_path}")

    # 8) gen_witness
    with logger.timed("gen_witness") as info:
        witness_path = os.path.join(wd, "witness.json")
        ok = run_with_loop(ezkl.gen_witness, data=input_json, model=compiled_path, output=witness_path)
        if not ok:
            raise RuntimeError("gen_witness failed")
        info["witness_path"] = witness_path
        
        # Torch peek for numerical accuracy comparison
        with torch.no_grad():
            opinions, mask = _gen_synthetic_opinions(max_opinions)
            opinions_b, mask_b = opinions.unsqueeze(0), mask.unsqueeze(0)
            opinions_flat = opinions_b.flatten(start_dim=1)
            mask_flat = mask_b.flatten(start_dim=1)
            combined_input = torch.cat([opinions_flat, mask_flat], dim=1)
            fused_t, rep_t = EBslFusionModule(max_opinions=max_opinions).eval()(combined_input)
        info["torch_fused"] = fused_t[0].detach().cpu().numpy().tolist()
        info["torch_rep"] = rep_t[0].detach().cpu().numpy().tolist()
        logger.ok(f"Witness generated -> {witness_path}")
        logger.info("Torch fused: " + json.dumps(info["torch_fused"], indent=2))
        logger.info("Torch rep:   " + json.dumps(info["torch_rep"], indent=2))

    # 9) mock
    with logger.timed("mock"):
        ok = run_with_loop(ezkl.mock, witness=witness_path, model=compiled_path)
        if not ok:
            raise RuntimeError("mock failed")
        logger.ok("Mock successful")

    # 10) prove
    with logger.timed("prove") as info:
        proof_path = os.path.join(wd, "proof.pf")
        ok = run_with_loop(
            ezkl.prove,
            witness=witness_path, model=compiled_path, pk_path=pk_path,
            proof_path=proof_path, srs_path=srs_path, proof_type="single"
        )
        if not ok:
            raise RuntimeError("prove failed")
        info["proof_path"] = proof_path
        try:
            proof_size = os.path.getsize(proof_path)
            info["proof_size_bytes"] = proof_size
            logger.ok(f"Proof generated -> {proof_path} ({proof_size:,} bytes, {proof_size/1024:.1f} KB)")
        except Exception:
            logger.ok(f"Proof generated -> {proof_path}")

    # 11) verify
    with logger.timed("verify") as info:
        ok = run_with_loop(ezkl.verify, proof_path=proof_path, settings_path=settings_path, vk_path=vk_path, srs_path=srs_path)
        info["verified"] = bool(ok)
        if ok:
            logger.ok("Proof verified ✅")
        else:
            logger.error("Proof verification failed ❌")
    
    return proof_path, info["verified"]

## 11. Demo Execution

In [ ]:
# Initialize logger
logger = Logger(verbose=True)

# Configuration
max_opinions = 8  # Smaller for demo
zk_strategy = "balanced"
skip_calibration = False  # Calibration required for proper functioning

logger.banner("EBSL + EZKL Demo Pipeline")
print(f"Configuration: max_opinions={max_opinions}, strategy={zk_strategy}, skip_calibration={skip_calibration}")

### Network Topology and ZK Circuit Visualization

In [ ]:
# Visualize the trust network topology
logger.info("Generating network topology visualization...")
trust_matrix = visualizer.create_network_topology(
    num_nodes=max_opinions,
    title=f"Trust Network Topology ({max_opinions} agents)"
)

# Show the ZK circuit concept
logger.info("Generating ZK circuit concept visualization...")
visualizer.visualize_zk_circuit_concept(
    "EBSL Zero-Knowledge Circuit Architecture"
)

In [ ]:
# Step 1: Property-based correctness test
try:
    run_property_based_correctness_test(logger)
except Exception as e:
    logger.error(f"Property test failed: {e}")

In [ ]:
# Step 2: Performance analysis
try:
    run_comparative_performance_analysis(logger, skip_plots=False)
except Exception as e:
    logger.error(f"Performance analysis failed: {e}")

In [ ]:
# Step 3: ZKML Pipeline - Initial Setup
try:
    onnx_path, settings_path = run_zkml_pipeline_with_ebsl(
        logger, 
        max_opinions=max_opinions,
        zk_strategy=zk_strategy,
        skip_calibration=skip_calibration
    )
    logger.ok("Initial pipeline setup completed")
except Exception as e:
    logger.error(f"Initial setup failed: {e}")
    raise

In [ ]:
# Step 4: Continue Pipeline - Compilation
try:
    compiled_path, srs_path, input_json = continue_zkml_pipeline(
        logger, onnx_path, settings_path, max_opinions, skip_calibration
    )
    logger.ok("Pipeline compilation completed")
except Exception as e:
    logger.error(f"Pipeline compilation failed: {e}")
    raise

In [ ]:
# Step 5: Complete Proof Generation
try:
    proof_path, verified = complete_proof_generation(
        logger, compiled_path, srs_path, input_json, settings_path, max_opinions
    )
    if verified:
        logger.ok("🎉 Complete EBSL+EZKL pipeline successful!")
    else:
        logger.error("❌ Proof verification failed")
except Exception as e:
    logger.error(f"Proof generation failed: {e}")
    raise

### Reputation Analysis and Visualization

In [ ]:
# Generate and visualize reputation scores
try:
    logger.info("Generating reputation analysis...")
    
    # Create sample opinions for reputation calculation
    np.random.seed(42)  # For reproducible results
    num_agents = max_opinions
    
    # Generate diverse opinion profiles
    beliefs = np.random.beta(2, 3, num_agents)  # Skewed towards lower belief
    disbeliefs = np.random.beta(1.5, 4, num_agents) * (1 - beliefs)
    uncertainties = 1 - beliefs - disbeliefs
    base_rates = np.random.uniform(0.3, 0.7, num_agents)  # Reasonable base rates
    
    opinions_tensor = torch.tensor(np.column_stack([beliefs, disbeliefs, uncertainties, base_rates]), dtype=torch.float32)
    
    # Calculate reputation scores
    reputation_scores = []
    for i in range(num_agents):
        individual_opinion = opinions_tensor[i:i+1]  # Single opinion
        reputation = EBSLAlgorithm.calculate_reputation(individual_opinion).item()
        reputation_scores.append(reputation)
    
    # Visualize reputation scores
    visualizer.plot_reputation_evolution(
        reputation_scores,
        list(range(num_agents)),
        f"Agent Reputation Scores (n={num_agents})"
    )
    
    # Create a comprehensive trust network with calculated reputations
    logger.info("Creating comprehensive trust analysis...")
    
    # Use reputation scores to create weighted trust matrix
    trust_matrix_weighted = np.zeros((num_agents, num_agents))
    for i in range(num_agents):
        for j in range(num_agents):
            if i != j:
                # Trust is influenced by target's reputation and relationship strength
                base_trust = trust_matrix[i, j] if 'trust_matrix' in locals() else np.random.beta(2, 5)
                reputation_factor = reputation_scores[j]
                trust_matrix_weighted[i, j] = base_trust * (0.5 + 0.5 * reputation_factor)
            else:
                trust_matrix_weighted[i, j] = 1.0  # Self-trust
    
    # Visualize the reputation-weighted network
    visualizer.create_network_topology(
        num_nodes=num_agents,
        trust_matrix=trust_matrix_weighted,
        title=f"Reputation-Weighted Trust Network ({num_agents} agents)"
    )
    
    logger.ok(f"Reputation analysis complete. Average reputation: {np.mean(reputation_scores):.3f}")
    
except Exception as e:
    logger.error(f"Reputation analysis failed: {e}")

In [ ]:
# Step 6: Generate Report
try:
    report_path = os.path.join("zkml_artifacts", "run_report.json")
    logger.dump_report(report_path)
    
    # Summary
    total_time = sum(step.seconds for step in logger.steps)
    success_count = sum(1 for step in logger.steps if step.ok)
    total_steps = len(logger.steps)
    
    logger.banner("Pipeline Summary")
    print(f"Total execution time: {total_time:.2f} seconds")
    print(f"Successful steps: {success_count}/{total_steps}")
    print(f"Success rate: {success_count/total_steps*100:.1f}%")
    
    if verified:
        print("\n✅ EBSL+EZKL pipeline completed successfully!")
        print("🔒 Zero-knowledge proof generated and verified")
        print("📊 EBSL fusion algorithm working in ZK environment")
    else:
        print("\n⚠️  Pipeline completed with verification issues")
        
except Exception as e:
    logger.error(f"Report generation failed: {e}")